## Deep Learning Approach

In this challenge, I started to use some idea from text generation to train my system. Because, text generation is based on previous elements and it generates the most probable next word. (Actually, it is depend on the _Temperature_ value, however, if you set this value like 0.4, it generate mostly most probable one.)

There are different implementations for text generation, however, most of them is based on char-rnn. So that, we can not use these versions for word level based prediction. If we can use char based generation, [this implemantation](https://github.com/minimaxir/textgenrnn) will be one of the best option. Because, it is well optimized for memory usage and speed.

_One last thing that textgenrnn can do that most char-rnn implementations can’t is generate a word level model (thanks to Keras’s tokenizers), where the model uses the n previous words/punctuation to predict the next word/punctuation._ [Source](https://minimaxir.com/2018/05/text-neural-networks/)

So that, I will choose [this blogpost](https://medium.com/@david.campion/text-generation-using-bidirectional-lstm-and-doc2vec-models-1-3-8979eb65cb3a) as my intuition. I will change and tweak the methods according to my own need.


**DATASET**

I have tried [the blog authorship corpus.](http://u.cs.biu.ac.il/~koppel/BlogCorpus.htm) This corpus includes .xml files for blogpost. I thought that it can represent real human conservation.

Firstly, I need to convert to .txt files. Also, when I check the dataset, I realize that, if the writer is younger than 21, it contains too much slang and typo. So that, I will use blogpost whose writers are older than 21. 

For xml manipulation, BeatifulSoup is one of the best option.

In [ ]:
from bs4 import BeautifulSoup
import glob
import os

xml_root_dir = glob.glob(os.path.join('./blogs/', '*xml'))

for xml_file in xml_root_dir:
    
    if (int((xml_file.split('/')[-1]).split('.')[2]) > 21):
    
        try:
            with open(xml_file) as fp:
                soup = BeautifulSoup(fp, "lxml")

                text_file = open('./blogs_txt/' + (xml_file.split('/')[-1])[:-4] + ".txt", 'w')
                posts = soup.findAll('post')
                for single_post in posts:
                    text_file.write(single_post.text)

                text_file.close()
        except UnicodeDecodeError:
            pass
        

When I train the LSTM system with this dataset, results are not satisfied. It mostly generates same words. I use simple LSTM model. Maybe, if we can clear more and train with more deep architecture, we can get good results. 

_Ps. I have trained it for 30 minutes. I don't supply the training result. Because I want to keep clear this notebook_

Also, I have tried [Cornell Movie Dialog Corpus.](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) We need to preprocess this txt file.

In [1]:
with open('./cornell_movie_quotes_corpus/moviequotes.scripts.txt', encoding = "ISO-8859-1") as f:
    content = f.readlines()
content = [x.strip() for x in content]     

In [2]:
with open('cornell_clean.txt', 'w') as f:
    counter = 0
    for single_content in content:
        single_sentence = (single_content.split('+++$+++')[-1])
        f.write(single_sentence)
        counter += 1
        if (counter == 5000): # How many file will be taken
            break

f.close()

-----

In [3]:
from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, GRU
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.metrics import categorical_accuracy
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D
from keras.models import load_model

#import spacy, and spacy french model
# spacy is used to work on text
import spacy
nlp = spacy.load('en')

#import other libraries
import numpy as np
import random
import sys
import glob
import re
import os
import time
import codecs
import collections
from six.moves import cPickle

/Users/herdogan/anaconda3/envs/pyannote/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
save_dir = 'save' # To store trained DL model
file_list = glob.glob(os.path.join('./', 'cornell_clean.txt'))
vocab_file = os.path.join(save_dir, "words_vocab.pkl")
sequences_step = 1 # Step to create sequences

I will use [Spacy](https://spacy.io) to tokenize my dataset. If we use _Keras_ tokenizer, we can not work on word-level creation or prediction.

In [5]:
def create_wordlist(doc):
    """Append all words in one .txt files into one list.
    
    Arguments:
    doc: The document's directory"""
    
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n",'\u2009','\xa0', 'urlLink'):
            wl.append(word.text.lower())
    return wl

Now, we can create a list which inclued all words in our dataset.

In [6]:
wordlist = []

for input_file in file_list:
    
    # Read data
    with codecs.open(input_file, "r") as f:
        data = f.read()
        
    pattern = re.compile('[^a-zA-Z\d\s]')
    data_revise = pattern.sub('', data)
        
    # Create sentences with spacy tokenizer.
    doc = nlp(data)
    wl = create_wordlist(doc)
    wordlist = wordlist + wl

We need to create a dictionary, each specific word has own specific index. 

In [7]:
# Count the number of words
word_counts = collections.Counter(wordlist)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

# Size of vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)

# Save the words and vocabulary as a pickle file
with open(os.path.join(vocab_file), 'wb') as f:
    cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  6050


In [8]:
"""We need to create two different list. One list include the previous words, another list inclued the next word."""

seq_length = 15

sequences = []
next_words = []
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 68710


In [9]:
"""This gif can give a idea to how system predict the next word. This gif was created for my music generation project."""

from IPython.display import HTML
HTML('<img src="https://s1.gifyu.com/images/try-2018-04-15-03.42.07.gif">')

In [10]:
"""We can not use this type of array directly. So that, we have to modify this data to use with LSTM. We need 
to convert into one-hot vector type array. 

List which includes previous words should have a dimension as number of sequences, number of words in sequences,
number of words in vocabulary. The other list should have a dimension as number of sequences, 
number of words in vocabulary."""

X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1

In [11]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu", return_sequences=True, 
                                 kernel_initializer='random_normal',
                                    bias_initializer='random_normal'),
                            input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(rnn_size, activation="relu",
                                kernel_initializer='random_normal',
                                    bias_initializer='random_normal')))
    model.add(Dropout(0.3))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    print("model built!")
    return model

In [12]:
rnn_size = 32 # size of RNN
learning_rate = 0.001 

md = bidirectional_lstm_model(seq_length, vocab_size)
md.summary()

Build LSTM model.
model built!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 15, 64)            1557248   
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 64)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6050)              393250    
_________________________________________________________________
activation_1 (Activation)    (None, 6050)              0         
Total params: 1,975,330
Trainable params: 1,975,330
Non-trainable params: 0
___________________________________

In [13]:
batch_size = 32 # minibatch size
num_epochs = 2 # number of epochs

callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath=save_dir + "/" + 'my_model_gen_sentences.{epoch:02d}-{val_loss:.2f}.hdf5',\
                           monitor='val_loss', verbose=0, mode='auto', period=2)]

history = md.fit(X, y,
                 batch_size=batch_size,
                 shuffle=True,
                 epochs=num_epochs,
                 callbacks=callbacks,
                 validation_split=0.1)

md.save(save_dir + "/" + 'my_model_generate_sentences.h5')

Train on 61839 samples, validate on 6871 samples
Epoch 1/2
61839/61839 [==============================] - 494s 8ms/step - loss: 6.2289 - categorical_accuracy: 0.0711 - val_loss: 5.7886 - val_categorical_accuracy: 0.0688
Epoch 2/2
61839/61839 [==============================] - 542s 9ms/step - loss: 5.9084 - categorical_accuracy: 0.0771 - val_loss: 5.7305 - val_categorical_accuracy: 0.0956


In [14]:
save_dir = "./save/"

# Load vocabulary
print("loading vocabulary...")
vocab_file = os.path.join(save_dir, "words_vocab.pkl")

with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        words, vocab, vocabulary_inv = cPickle.load(f)

vocab_size = len(words)

# Load the model
print("loading model...")
model = load_model(save_dir + "/" + 'my_model_generate_sentences.h5')

loading vocabulary...
loading model...


In [15]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [16]:
# Iniatate sentence
seed_sentences = "i will need"
generated = ''
sentence = []
for i in range (seq_length):
    sentence.append("a")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)
print('Generating text with the following seed: "' + ' '.join(sentence) + '"')

print ()

Generating text with the following seed: "a a a a a a a a a a a a i will need"



In [17]:
words_number = 10
# Generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1.
    #print(x.shape)

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.2)
    next_word = vocabulary_inv[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

print(generated)

a a a a a a a a a a a a i will need .   .     .   ?    


As you can see, it can not create reasonable result with simple DNN model. Also, I have to clean the dataset with better methods to get better result. So that, I have tried to n-grams model. They works better.

---
## N-GRAMS

[Idea](https://web.stanford.edu/~jurafsky/slp3/slides/LM_4.pdf)

[Some Code](https://github.com/rikenshah/predict-next-word)


I want to create the system which has low memory usage and need low CPU power. So that, I will try [python containers](https://docs.python.org/3/library/collections.html) and compare them.

Firstly, I will start with how we can use txt file to create bi-grams. I will use Cornell Movie Quote Dataset as my text generation system.

In [53]:
import re
from nltk import ngrams
from collections import Counter
import nltk

"""I just want to alpha-numeric characters."""

pattern = re.compile('[^a-zA-Z\d\s]')
data_revise = pattern.sub('', data)

In [54]:
data_revise = data_revise.lower()
text = data_revise.split()
nonPunct = re.compile('.*[A-Za-z0-9].*')  # must contain a letter or digit
filtered = [w for w in text if nonPunct.match(w)]
unigramsCounter = Counter(filtered)
# counts

In [55]:
% time

bigrams = ngrams(data_revise.split(), 2)

bigramsCounter = Counter(bigrams)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs


In [56]:
% time

tokens = nltk.word_tokenize(data_revise)

#Create your bigrams
bgs = nltk.bigrams(tokens)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(bgs)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs


There is not much difference between these 2 methods in terms of memory usage and CPU. I will use Counter.

In [68]:
%time
import pickle
from collections import OrderedDict

# We need to save Counter as a pickle to use later.
pickleDumps = "pickleDumps/"
conditionalProbabilityFile = pickleDumps+"conditionalProbabilityDictBigram.p"
bigramsCounterPath = pickleDumps+"bigramsCounter.p"


conditionalProbabilityDict = OrderedDict() # Key: Bigram, Value: Prob. of bigram

for words, count in bigramsCounter.items():
    firstWord = words[0]
    secondWord = words[1]
    count = count
    cProb = count * 1.0 / unigramsCounter[firstWord]
    conditionalProbabilityDict[firstWord+" "+secondWord] = cProb


CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


In [67]:
file = open(bigramsCounterPath,"wb")
pickle.dump(bigramsCounter,file)
file = open(conditionalProbabilityFile,"wb")
pickle.dump(conditionalProbabilityDict,file)

I can not enter input via Jupyter notebook. So we need to my python script. However, I will give some example from system.

**Examples**

Enter a word to predict its next probable words (':)' for stopping) : i g
i got : 0.016574585635359115

i guess : 0.011740331491712707

i get : 0.011740331491712707

Enter a word to predict its next probable words (':)' for stopping) : you c
you can : 0.0210688591983556

you cant : 0.012846865364850977

you could : 0.008735868448098663

Enter a word to predict its next probable words (':)' for stopping) : for
for the : 0.13903743315508021

for a : 0.11497326203208556

for you : 0.09358288770053476


Enter a word to predict its next probable words (':)' for stopping) : for in
for information : 0.00267379679144385

-----
As you can see, results are not so satisfied. I want to improve it. So that, I will use directly [COCA Corpus](https://www.ngrams.info/samples_coca1.asp) which contains the 1,000,000 most frequent 2, 3, 4, and 5-grams. I will use non-case sensitive corpus. _(To download it, you need to register with your e-mail. So, I cannot share wget download script to this Corpus)_

In [69]:
with open('./w2_.txt', encoding = "ISO-8859-1") as f:
    content = f.readlines()
content = [x.strip() for x in content]     

In [70]:
%time

import pickle, os
from collections import OrderedDict

corpusPath = "./"
fileName = "w2_.txt"
pickleDumps = "pickleDumps/"
conditionalProbabilityFile = pickleDumps+"conditionalProbabilityDictBigram.p"
bigramsCounterPath = pickleDumps + "bigramsCounter.p"

with open(corpusPath+fileName, encoding = "ISO-8859-1") as f:
    lines = f.readlines()

bigramsCounter = {} # To store bigrams count. Key: single word, value: how many
unigramsCounter = {} # To store unigrams count. Key: word pair, value: how many

for line in lines:
    
    # Removing \n and \r that were due to readline and splitting by tab
    singleLine = line.replace('\r','').replace('\n','').split('\t')
    
    bigramsCounter[singleLine[1], singleLine[2]] = int(singleLine[0])
    
    # If key does not exist, we need to create key, value pair with inital values
    # If key exists then add the count of that unigram
    if singleLine[1] in unigramsCounter.keys():
        unigramsCounter[singleLine[1]] += int(singleLine[0])
    else:
        unigramsCounter[singleLine[1]] = int(singleLine[0])

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


In [72]:
%time
import pickle

conditionalProbabilityDict = OrderedDict() # key:bigram , value:probability
for words, count in bigramsCounter.items():
    firstWord = words[0]
    secondWord = words[1]
    count = count
    cProb = count * 1.0 / unigramsCounter[firstWord]
    conditionalProbabilityDict[firstWord+" "+secondWord] = cProb

# print conditionalProbabilityDict
file = open(conditionalProbabilityFile,"wb")
pickle.dump(conditionalProbabilityDict,file)

file = open(bigramsCounterPath,"wb")
pickle.dump(bigramsCounter,file)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


**Memory Usage for Pickle Files**

If we use orderedDict to store bigrams, pickleDumps folder takes 83.8 MB memory. 

If use Counter to store bigrams, pickleDumps folder takes 71.3 MB memory.

**Speed**

I have tried both store method with different prediction string, there is no sufficient difference between them. However, I need to test with at least 10000 different scripts to understand difference.

Mostly, it can give the result in 100ms-150ms.

**Examples**


Enter a word to predict its next probable words (':)' for stopping) : for
for the : 0.18615500817070693

for a : 0.09376909228897273

for example : 0.025409373405987343

Enter a word to predict its next probable words (':)' for stopping) : for in
for instance : 0.008036617330354772

for information : 0.0013408959057072335

for in : 0.0010947629860785409

Enter a word to predict its next probable words (':)' for stopping) : can we g
we got : 0.012135281736282812

we get : 0.010222156277533184

we go : 0.008274891311626734

----

To improve result, I want to implement trigrams. However, because of limited time, I can not create combination of both method.

**3-GRAM**

In [73]:
%time

import pickle, os
from collections import OrderedDict

corpusPath = "./"
fileName = "w3_.txt"
pickleDumps = "pickleDumps/"
conditionalProbabilityFile = pickleDumps+"conditionalProbabilityDictTrigram.p"
trigramsCounterPath = pickleDumps + "trigramsCounter.p"

with open(corpusPath+fileName, encoding = "ISO-8859-1") as f:
    lines = f.readlines()

trigramsCounter = {} # To store bigrams count. Key: single word, value: how many
unigramsCounter = {} # To store unigrams count. Key: word pair, value: how many

for line in lines:
    
    # Removing \n and \r that were due to readline and splitting by tab
    singleLine = line.replace('\r','').replace('\n','').split('\t')
    
    trigramsCounter[singleLine[1], singleLine[2], singleLine[3]] = int(singleLine[0])
    
    # If key does not exist, we need to create key, value pair with inital values
    # If key exists then add the count of that unigram
    if singleLine[1] in unigramsCounter.keys():
        unigramsCounter[singleLine[1]] += int(singleLine[0])
    else:
        unigramsCounter[singleLine[1]] = int(singleLine[0])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [74]:
%time
import pickle

conditionalProbabilityDict = OrderedDict() # key:bigram , value:probability
for words, count in trigramsCounter.items():
    firstWord = words[0]
    secondWord = words[1]
    thirdWord = words[2]
    count = count
    cProb = count * 1.0 / unigramsCounter[firstWord]
    conditionalProbabilityDict[firstWord+" "+secondWord + " " +thirdWord] = cProb

# print conditionalProbabilityDict
file = open(conditionalProbabilityFile,"wb")
pickle.dump(conditionalProbabilityDict,file)

file = open(trigramsCounterPath,"wb")
pickle.dump(trigramsCounter,file)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


**Examples**

Enter a word to predict its next probable words (':)' for stopping) : can i t

Bigrams prediction:

i think : 0.08613532912264124

i thought : 0.014199842979509856

i told : 0.006020473347966184

____________________

Trigrams prediction:

can i tell : 0.0009584862682746684

can i take : 0.00035332434987379933

can i talk : 0.00034392742567502805-

--------------

Enter a word to predict its next probable words (':)' for stopping) : should we go to

Bigrams prediction:

to the : 0.10137702785084556

to be : 0.06316283346651083

to a : 0.02362025832529113

____________________

Trigrams prediction:

go to the : 0.08628557038529337

go to a : 0.023319063208259883

go to school : 0.010480987156887434

-----

**Speed**

Mostly, it can give the result in 1500 ms. So, it is somewhat slow procedure. However, we can use different methods to store and call the words. But, I do not know so much things about data structures. 

---
**With More Time**

- Implement the SQLITE type storage to get results faster. http://zetcode.com/db/sqlitepythontutorial/

- Calculate the perplexity for test set.

- Implement the linear interpolation to guess next word.

- Try to implement with low-level language.